### Animated GIF showing the rate of agriculture

In [1]:
# importing the necessary libraries
import glob
import os
import rasterio
from rasterio.mask import mask
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap
from IPython.display import HTML
from matplotlib.patches import Patch

In [2]:
# defining the paths
shapefile_path = "/shared_space/BrazilSPEI/Maranhao_boundaries_EPSG/maranhao_boundary.shp"
agri_dir = "/shared_space/BrazilSPEI/MODIS/Reclass_Data"
output_gif = "maranhao_agriculture_animation.gif"

In [3]:
# animation settings
fps = 2                # frames per second or the speed
dpi = 150              # resolution
interval = 600         # milliseconds per frame

In [4]:
# loading the Maranhao boundary
print("Loading Maranhão shapefile...")
ma = gpd.read_file(shapefile_path)
ma.crs = "EPSG:4326"
print(f"Shapefile loaded: {ma.crs}")

Loading Maranhão shapefile...
Shapefile loaded: EPSG:4326


In [5]:
# loading all the agriculture rasters
print("Loading agriculture cover rasters...")
agri_paths = sorted(glob.glob(os.path.join(agri_dir, "*_agri.tif")))

if len(agri_paths) == 0:
    print(f"ERROR: No agricultural rasters found in {agri_dir}")
    exit(1)

print(f"Found {len(agri_paths)} agricultural rasters")

Loading agriculture cover rasters...
Found 24 agricultural rasters


In [6]:
# extracting the years from filenames
years = []
for path in agri_paths:
    fname = os.path.basename(path)
    # extracting 4-digit year
    digits = ''.join(filter(str.isdigit, fname))
    if len(digits) >= 4:
        year = int(digits[-4:]) 
        years.append(year)
    else:
        years.append(None)

In [9]:
# loading raster data
agri_arrays = []
extent = None
raster_crs = None

for i, path in enumerate(agri_paths):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(float)
        agri_arrays.append(arr)
        
        if extent is None:
            bounds = src.bounds
            extent = (bounds.left, bounds.right, bounds.bottom, bounds.top)
            raster_crs = src.crs
        
        # calculating the agricultural area for this year
        agri_pixels = np.sum(arr == 1)
        agri_area_km2 = agri_pixels * 0.25  # each pixel is 500m x 500m = 0.25 km²
        
        if years[i]:
            print(f"  Year {years[i]}: {agri_area_km2:,.0f} km² agricultural land")

print(f"Loaded {len(agri_arrays)} rasters")
print(f"CRS: {raster_crs}")
print(f"Shape: {agri_arrays[0].shape}")
print(f"Extent: {extent}")

  Year 2001: 2,605 km² agricultural land
  Year 2002: 2,472 km² agricultural land
  Year 2003: 2,642 km² agricultural land
  Year 2004: 2,764 km² agricultural land
  Year 2005: 2,976 km² agricultural land
  Year 2006: 3,203 km² agricultural land
  Year 2007: 3,357 km² agricultural land
  Year 2008: 3,726 km² agricultural land
  Year 2009: 3,899 km² agricultural land
  Year 2010: 3,956 km² agricultural land
  Year 2011: 4,225 km² agricultural land
  Year 2012: 4,402 km² agricultural land
  Year 2013: 4,539 km² agricultural land
  Year 2014: 4,611 km² agricultural land
  Year 2015: 4,853 km² agricultural land
  Year 2016: 4,524 km² agricultural land
  Year 2017: 4,874 km² agricultural land
  Year 2018: 5,334 km² agricultural land
  Year 2019: 5,190 km² agricultural land
  Year 2020: 5,173 km² agricultural land
  Year 2021: 5,130 km² agricultural land
  Year 2022: 5,260 km² agricultural land
  Year 2023: 5,922 km² agricultural land
  Year 2024: 6,064 km² agricultural land
Loaded 24 raster

In [10]:
# reprojecting the shapefile if needed
if raster_crs != ma.crs:
    print(f"\nReprojecting shapefile from {ma.crs} to {raster_crs}")
    ma = ma.to_crs(raster_crs)

In [14]:
# creating the animation
print("Creating animation...")

# color map: gray for non-forest (0), dark green for forest (1)
cmap = ListedColormap(["#e6e6e6", "#d4a017"])

# creating the figure
fig, ax = plt.subplots(figsize=(8, 10))
fig.patch.set_facecolor('white')

# displaying first frame
im = ax.imshow(
    agri_arrays[0], 
    cmap=cmap, 
    vmin=0, 
    vmax=1, 
    extent=extent,
    interpolation='nearest'
)

title = ax.text(
    0.5, 1.07,
    f"Maranhão Agricultural Cover — {years[0]}",
    transform=ax.transAxes,
    ha="center", 
    fontsize=20, 
    fontweight="bold"
)

agri_pixels_0 = np.sum(agri_arrays[0] == 1)
agri_area_0 = agri_pixels_0 * 0.25
area_text = ax.text(
    0.5, 1.02,
    f"Agricultural Area: {agri_area_0:,.0f} km²",
    transform=ax.transAxes,
    ha="center",
    fontsize=14,
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
)

# drawing the state boundary
ma.boundary.plot(ax=ax, color="black", linewidth=1.5)

# setting the map extent and setting axes
ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])
ax.set_xticks([])
ax.set_yticks([])
ax.set_aspect('equal')

# adding legend
legend_elements = [
    Patch(facecolor='#d4a017', label='Agricultural Land'),
    Patch(facecolor='#e6e6e6', label='Non-agricultural Land')
]
ax.legend(
    handles=legend_elements, 
    loc='lower right',
    frameon=True,
    fancybox=True,
    shadow=True
)

# updating the function
def update(i):
    # updating the raster
    im.set_data(agri_arrays[i])
    
    # updating the title
    title.set_text(f"Maranhão Agricultural Cover — {years[i]}")
    
    # updating the area text
    agri_pixels = np.sum(agri_arrays[i] == 1)
    agri_area = agri_pixels * 0.25
    area_text.set_text(f"Agricultural Area: {agri_area:,.0f} km²")
    
    return [im, title, area_text]

print(f"Generating animation with {len(agri_arrays)} frames...")

anim = FuncAnimation(
    fig, 
    update,
    frames=len(agri_arrays),
    interval=interval,
    blit=True,
    repeat=True
)

# saving it as a gif
anim.save(
    output_gif, 
    writer='pillow', 
    fps=fps, 
    dpi=dpi
)

print(f"Output file: {output_gif}")
print(f"Duration: ~{len(agri_arrays) / fps:.1f} seconds")
print(f"Resolution: {dpi} DPI")

plt.close()


HTML(anim.to_html5_video())

Creating animation...
Generating animation with 24 frames...
Output file: maranhao_agriculture_animation.gif
Duration: ~12.0 seconds
Resolution: 150 DPI
